In [29]:
import pandas as pd
import os
from datetime import datetime

Iterate through the raw files and create a master file:

In [30]:
directory = os.getcwd() + "/spreadsheets/"
df2 = pd.DataFrame()
for filename in os.listdir(directory):
     if filename.endswith(".xlsx"):
        df = pd.read_excel(directory + filename)
        df = df[['   Rank', 'Emp.No.','Org. Unit', 'SEX', 'Race', 'Nationality', ' Marital St.', 'Certificate','Date of Birth', 'Date of Enlistment']]
        date = datetime.strptime(filename[0:-5], '%b %y')
        df['Date of Headcount'] = date.date()
        df2 = df2.append(df)
df2.to_csv("df2.csv",index=False)

In [31]:
df2.rename(columns={'   Rank':'Rank', 'Emp.No.':'Staff ID', ' Marital St.':'Marital Status'}, inplace=True)
df2['Date of Headcount'] = pd.to_datetime(df2['Date of Headcount'])
df2['Date of Headcount'] = (df2['Date of Headcount'] + pd.offsets.MonthEnd(n=2)).apply(lambda dt: dt.replace(day=1))

#Headcount is at last day of every month, we coerce to first day of next month

In [32]:
df2 = df2[df2['Org. Unit']!='APF HQ Recruits Branch']
df2 = df2[df2['Org. Unit']!='Planning']
df2 = df2[df2['Org. Unit']!='Manpower Planning Officer']

Merge education rank:

In [33]:
education = pd.read_csv("educationrank.csv")
master = pd.merge(df2, education, how="left", left_on='Certificate', right_on='Certificate text')
master.rename(columns={'SEX':'Sex', 'Ranking':'Education Rank'}, inplace=True)
master.drop(['Certificate','Certificate text'], axis=1, inplace=True)

Merge org unit groups:

In [34]:
orgunits = pd.read_csv("orgunits.csv")
master = pd.merge(master, orgunits, how="left", on="Org. Unit")
master.rename(columns={'Org. Unit Grouping':'Org Unit Group'}, inplace=True)
master.drop(['Org. Unit'], axis=1, inplace=True)

In [37]:
#master['Length of Service'] = master['Date of Headcount'] - master['Date of Enlistment']
#master['Age'] = master['Date of Headcount'] - master['Date of Birth']
master.dropna(subset=['Org Unit Group'],inplace=True)
master.drop_duplicates(inplace=True)
master.to_csv("master.csv", index=False)

In [36]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193776 entries, 0 to 193775
Data columns (total 11 columns):
Rank                  193776 non-null object
Staff ID              193776 non-null int64
Sex                   193776 non-null object
Race                  193776 non-null object
Nationality           193776 non-null object
Marital Status        193703 non-null object
Date of Birth         193776 non-null datetime64[ns]
Date of Enlistment    193776 non-null datetime64[ns]
Date of Headcount     193776 non-null datetime64[ns]
Education Rank        193768 non-null float64
Org Unit Group        191686 non-null object
dtypes: datetime64[ns](3), float64(1), int64(1), object(6)
memory usage: 17.7+ MB


In [38]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127111 entries, 0 to 193775
Data columns (total 11 columns):
Rank                  127111 non-null object
Staff ID              127111 non-null int64
Sex                   127111 non-null object
Race                  127111 non-null object
Nationality           127111 non-null object
Marital Status        127098 non-null object
Date of Birth         127111 non-null datetime64[ns]
Date of Enlistment    127111 non-null datetime64[ns]
Date of Headcount     127111 non-null datetime64[ns]
Education Rank        127104 non-null float64
Org Unit Group        127111 non-null object
dtypes: datetime64[ns](3), float64(1), int64(1), object(6)
memory usage: 11.6+ MB
